In [1]:
import pandas as pd
import numpy as np
import operator

import sklearn
from sklearn.svm import SVR  
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
dataset = pd.read_csv("Preprocessed Dataset.csv")
dataset.head()

,alarm_id,customer_id,hostname,fault_severity,device_contract_level,status,created_at,fault_description,file_name,impact,...,features,fingerprint,hw,last_reset,product_family,product_type,reset_reason,sw,sw_type_y,sw_version
0,24600766,Bg5Zru3Rw4tzK0,Device_63_115_116_140,Critical,2,Success,2023-10-13 01 00 10.000000,"Due to issues like memory fragmentation, somet...",116_FXS2311Q3UZ_Bg5Zru3Rw4tzK0_1697158810445_c...,Memory fragmentation leads crypto to fail and ...,...,NTPv4_IOS SNMP_Trap_memory Connection_and_Syst...,17_1 CAT9K_IOSXE SYSTEM IOS_XE 17_1_1 Cisco_Ca...,Cisco_Catalyst_9600_Series_Switches C9606R Cat...,2021_01_13T22:23:27,Cisco_Catalyst_9600_Series_Switches,Switches,PowerOn,17_1 CAT9K_IOSXE SYSTEM IOS_XE 17_1_1,IOS_XE,17_1_1
1,24600766,Bg5Zru3Rw4tzK0,Device_63_115_116_140,Critical,2,Success,2023-10-13 01 00 10.000000,"Due to issues like memory fragmentation, somet...",116_FXS2311Q3UZ_Bg5Zru3Rw4tzK0_1697158810445_c...,Memory fragmentation leads crypto to fail and ...,...,NTPv4_IOS Connection_and_System_Banners IKE_No...,16_12 CAT9K_IOSXE SYSTEM IOS_XE 16_12_4 Cataly...,Catalyst_9K_Series_Power_Supplies C9600_PWR_2K...,2020_10_13T14:21:48,Cisco_Catalyst_9600_Series_Switches,Switches,Critical_process_stack_mgr_fault_on_rp_0_0__rc...,16_12 CAT9K_IOSXE SYSTEM IOS_XE 16_12_4,IOS_XE,16_12_4
2,24600766,Bg5Zru3Rw4tzK0,Device_63_115_116_140,Critical,2,Success,2023-10-13 01 00 10.000000,"Due to issues like memory fragmentation, somet...",116_FXS2311Q3UZ_Bg5Zru3Rw4tzK0_1697158810445_c...,Memory fragmentation leads crypto to fail and ...,...,NTPv4_IOS SNMP_Trap_memory QoS_Infrastructure_...,17_3 CAT9K_IOSXE SYSTEM IOS_XE 17_3_4 Transcei...,Transceiver_Modules GLC_SX_MMD Transceiver_Mod...,2022_03_04T23:23:45,Cisco_Catalyst_9600_Series_Switches,Switches,ISSU,17_3 CAT9K_IOSXE SYSTEM IOS_XE 17_3_4,IOS_XE,17_3_4
3,24600766,Bg5Zru3Rw4tzK0,Device_63_115_116_140,Critical,2,Success,2023-10-13 01 00 10.000000,"Due to issues like memory fragmentation, somet...",116_FXS2311Q3UZ_Bg5Zru3Rw4tzK0_1697158810445_c...,Memory fragmentation leads crypto to fail and ...,...,NTPv4_IOS IOS_Spanning_Tree_Protocol_STP Conne...,16_12 CAT9K_IOSXE SYSTEM IOS_XE 16_12_4 Transc...,Transceiver_Modules SFP_10G_LR_S Catalyst_9K_S...,NaN,Cisco_Catalyst_9600_Series_Switches,Switches,NaN,16_12 CAT9K_IOSXE SYSTEM IOS_XE 16_12_4,IOS_XE,16_12_4
4,24600766,Bg5Zru3Rw4tzK0,Device_63_115_116_140,Critical,2,Success,2023-10-13 01 00 10.000000,"Due to issues like memory fragmentation, somet...",116_FXS2311Q3UZ_Bg5Zru3Rw4tzK0_1697158810445_c...,Memory fragmentation leads crypto to fail and ...,...,NTPv4_IOS SNMP_Trap_memory Connection_and_Syst...,17_3 CAT9K_IOSXE SYSTEM IOS_XE 17_3_3 Transcei...,Transceiver_Modules SFP_10G_LRM Transceiver_Mo...,2021_06_24T00:03:01,Cisco_Catalyst_9600_Series_Switches,Switches,Image_Install,17_3 CAT9K_IOSXE SYSTEM IOS_XE 17_3_3,IOS_XE,17_3_3


In [3]:
dataset.shape

(356, 42)

In [4]:
encoded_dataFrame = pd.DataFrame()
 
label_encoder = sklearn.preprocessing.LabelEncoder() 
for column in dataset.columns :
    if column != 'crashed':
        encoded_dataFrame[column] = label_encoder.fit_transform(dataset[column])

encoded_dataFrame['crashed'] = dataset['crashed']
encoded_dataFrame

,alarm_id,customer_id,hostname,fault_severity,device_contract_level,status,created_at,fault_description,file_name,impact,...,fingerprint,hw,last_reset,product_family,product_type,reset_reason,sw,sw_type_y,sw_version,crashed
0,0,0,2,0,0,0,0,0,0,1,...,77,73,21,1,0,35,18,0,17,0
1,0,0,2,0,0,0,0,0,0,1,...,23,47,14,1,0,11,6,0,6,1
2,0,0,2,0,0,0,0,0,0,1,...,110,90,88,1,0,31,23,0,22,0
3,0,0,2,0,0,0,0,0,0,1,...,30,109,102,1,0,38,6,0,6,0
4,0,0,2,0,0,0,0,0,0,1,...,91,105,36,1,0,32,22,0,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,5,0,0,0,0,0,5,1,5,0,...,3,54,7,0,0,35,1,0,1,0
352,5,0,0,0,0,0,5,1,5,0,...,12,1,102,0,0,35,5,0,5,0
353,5,0,0,0,0,0,5,1,5,0,...,24,70,37,0,0,32,6,0,6,0
354,5,0,0,0,0,0,5,1,5,0,...,48,25,1,0,0,35,13,0,12,0


In [5]:
correlation_dataFrame = encoded_dataFrame.corr()
correlation_dataFrame

,alarm_id,customer_id,hostname,fault_severity,device_contract_level,status,created_at,fault_description,file_name,impact,...,fingerprint,hw,last_reset,product_family,product_type,reset_reason,sw,sw_type_y,sw_version,crashed
alarm_id,1.000000,NaN,-0.484062,NaN,NaN,NaN,1.000000,2.271985e-01,1.000000,-0.270787,...,-1.207726e-01,-2.434602e-01,-2.399849e-02,-4.274306e-01,NaN,-1.742530e-01,-1.159337e-01,NaN,-1.243796e-01,2.329692e-01
customer_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hostname,-0.484062,NaN,1.000000,NaN,NaN,NaN,-0.484062,-4.693581e-01,-0.484062,0.559406,...,2.494983e-01,5.029524e-01,4.957730e-02,8.830079e-01,NaN,3.599807e-01,2.395017e-01,NaN,2.569497e-01,-4.812797e-01
fault_severity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_contract_level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
status,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
created_at,1.000000,NaN,-0.484062,NaN,NaN,NaN,1.000000,2.271985e-01,1.000000,-0.270787,...,-1.207726e-01,-2.434602e-01,-2.399849e-02,-4.274306e-01,NaN,-1.742530e-01,-1.159337e-01,NaN,-1.243796e-01,2.329692e-01
fault_description,0.227198,NaN,-0.469358,NaN,NaN,NaN,0.227198,1.000000e+00,0.227198,0.469358,...,-6.897481e-17,-1.833387e-16,-9.067012e-17,-1.765949e-16,NaN,-1.619973e-16,-5.051180e-16,NaN,-2.702970e-16,8.081714e-17
file_name,1.000000,NaN,-0.484062,NaN,NaN,NaN,1.000000,2.271985e-01,1.000000,-0.270787,...,-1.207726e-01,-2.434602e-01,-2.399849e-02,-4.274306e-01,NaN,-1.742530e-01,-1.159337e-01,NaN,-1.243796e-01,2.329692e-01
impact,-0.270787,NaN,0.559406,NaN,NaN,NaN,-0.270787,4.693581e-01,-0.270787,1.000000,...,2.494983e-01,5.029524e-01,4.957730e-02,8.830079e-01,NaN,3.599807e-01,2.395017e-01,NaN,2.569497e-01,-4.812797e-01


In [6]:
correlation_dataFrame['crashed']

alarm_id                 2.329692e-01
customer_id                       NaN
hostname                -4.812797e-01
fault_severity                    NaN
device_contract_level             NaN
status                            NaN
created_at               2.329692e-01
fault_description        8.081714e-17
file_name                2.329692e-01
impact                  -4.812797e-01
mgmt_system_addr                  NaN
mgmt_system_id                    NaN
mgmt_system_type                  NaN
remediation             -4.812797e-01
syslog_msg              -4.608195e-01
tac_severity                      NaN
afm_cust_id_fk                    NaN
prod_obj_id_fk           5.450457e-01
managed_ne_id           -4.812797e-01
product_id              -5.450457e-01
sw_type_x                         NaN
serial_number           -4.812797e-01
cli_status                        NaN
solution_id                       NaN
usecase_id               5.450457e-01
signature_uuid           8.081714e-17
primekey    

In [7]:
crashed_correlated_dataFrame = pd.DataFrame(correlation_dataFrame['crashed'])
crashed_correlated_dataFrame.dropna(inplace = True)
crashed_correlated_dataFrame

,crashed
alarm_id,2.329692e-01
hostname,-4.812797e-01
created_at,2.329692e-01
fault_description,8.081714e-17
file_name,2.329692e-01
impact,-4.812797e-01
remediation,-4.812797e-01
syslog_msg,-4.608195e-01
prod_obj_id_fk,5.450457e-01
managed_ne_id,-4.812797e-01


In [8]:
crashed_correlated_dict = dict()
columns_to_keep = list()

for index in range(crashed_correlated_dataFrame.shape[0]):
    key = crashed_correlated_dataFrame.iloc[index].name
    columns_to_keep.append(key)
    if key != 'crashed':
        crashed_correlated_dict[crashed_correlated_dataFrame.iloc[index].name] = crashed_correlated_dataFrame['crashed'].iloc[index]
        
crashed_correlated_dict_in_descending_order = dict(sorted(crashed_correlated_dict.items(), key = operator.itemgetter(1), reverse = True))
crashed_correlated_dict_in_descending_order

{'prod_obj_id_fk': 0.5450457022868392,
 'usecase_id': 0.5450457022868392,
 'primekey': 0.23696007171161598,
 'i_d': 0.23696007171161598,
 'alarm_id': 0.23296923529592925,
 'created_at': 0.23296923529592925,
 'file_name': 0.23296923529592925,
 'device_id': 0.21221688798345903,
 'last_reset': 0.04321553545356994,
 'fault_description': 8.08171360645626e-17,
 'signature_uuid': 8.08171360645626e-17,
 'sw': -0.10233972115429586,
 'fingerprint': -0.10532355506208228,
 'sw_version': -0.11736502533065689,
 'device_type': -0.2584837753976487,
 'cpy_key': -0.34977829123034615,
 'hw': -0.3765010858031829,
 'features': -0.37722047651911417,
 'syslog_msg': -0.4608194644914447,
 'hostname': -0.48127966679325834,
 'remediation': -0.48127966679325834,
 'impact': -0.4812796667932585,
 'managed_ne_id': -0.4812796667932585,
 'serial_number': -0.4812796667932585,
 'product_id': -0.5450457022868391,
 'product_family': -0.5450457022868391,
 'reset_reason': -0.7984304207914686}

In [9]:
columns_to_keep.remove('primekey')
columns_to_keep.remove('i_d')
columns_to_keep.remove('created_at')
columns_to_keep.remove('file_name')
columns_to_keep.remove('signature_uuid')
columns_to_keep.remove('sw')
columns_to_keep.remove('hw')
columns_to_drop = dataset.columns.difference(columns_to_keep)
print("Columns to be dropped - {}".format(columns_to_drop))

Columns to be dropped - Index(['afm_cust_id_fk', 'cli_status', 'created_at', 'customer_id',
       'device_contract_level', 'fault_severity', 'file_name', 'hw', 'i_d',
       'mgmt_system_addr', 'mgmt_system_id', 'mgmt_system_type', 'primekey',
       'product_type', 'signature_uuid', 'solution_id', 'status', 'sw',
       'sw_type_x', 'sw_type_y', 'tac_severity'],
      dtype='object')


In [10]:
from sklearn.model_selection import train_test_split

target_column = encoded_dataFrame['crashed']
feature_columns = encoded_dataFrame.drop(columns = columns_to_drop, axis = 1)
features_train, features_test, target_train, target_test = train_test_split(feature_columns, target_column, test_size = 0.4, random_state = 1)

In [11]:
mean_absolute_error_models_dict = dict()
mean_squared_error_models_dict = dict()
r2_score_models_dict = dict()

### SVR

In [12]:
SVR_model = SVR(kernel='linear') 
SVR_model.fit(features_train, target_train) 

SVR(kernel='linear')

In [13]:
target_predicted= SVR_model.predict(features_test)
target_predicted

array([ 0.06456357, -0.01851463,  0.03118004,  0.03324219,  0.0147166 ,
        0.05819222,  0.05044603, -0.06184713, -0.07901196, -0.00113839,
        0.96936287,  0.96813652,  0.89988327,  0.07265613,  0.06958467,
        0.05526236,  0.01393464,  0.01024023,  0.0685492 , -0.01853695,
        0.05526439,  0.97738964, -0.02187213,  0.06231176,  0.09230384,
       -0.00937023,  0.03324016,  0.07265335,  0.07790353,  0.02729216,
       -0.01867615,  0.09531915,  0.002901  ,  0.00924952,  0.04487632,
       -0.04765919,  0.9004477 ,  0.0435774 , -0.00596531,  0.04556154,
        0.02916506,  0.03118131,  0.01078841,  0.0012019 , -0.10001621,
        0.08149333,  0.98526929,  0.01393389, -0.02057758,  0.00507866,
        0.03118207,  0.09337248,  0.05526311, -0.08001532,  0.9503033 ,
        0.09945102, -0.02203185,  1.00459284, -0.07901475, -0.09546121,
        1.00065126,  0.0726541 ,  0.01039937,  0.00924749,  0.97738762,
        1.06621739, -0.0220311 ,  0.04555951,  0.07998218, -0.08

In [14]:
print("Mean Absolute Error: {}".format(mean_absolute_error(target_test, target_predicted)))
print("Mean Squared Error: {}".format(mean_squared_error(target_test, target_predicted)))

Mean Absolute Error: 0.046459810106820834
Mean Squared Error: 0.0031795857381291667


In [15]:
sklearn.preprocessing.minmax_scale(target_predicted, feature_range=(0, 1), axis=0, copy=False)

array([0.14112077, 0.06988444, 0.11249569, 0.1142639 , 0.09837893,
       0.13565759, 0.12901553, 0.03272851, 0.01801033, 0.08478389,
       0.91695101, 0.91589946, 0.85737495, 0.14805983, 0.14542617,
       0.13314534, 0.09770843, 0.09454062, 0.14453829, 0.0698653 ,
       0.13314708, 0.92383366, 0.06700552, 0.13918993, 0.16490698,
       0.07772541, 0.11426216, 0.14805744, 0.15255927, 0.10916198,
       0.06974594, 0.16749248, 0.08824751, 0.09369112, 0.12423972,
       0.04489411, 0.85785893, 0.12312594, 0.08064499, 0.12482726,
       0.11076792, 0.11249678, 0.09501066, 0.0867906 , 0.        ,
       0.15563738, 0.93059015, 0.09770779, 0.06811554, 0.09011477,
       0.11249743, 0.16582329, 0.13314599, 0.01714999, 0.90060816,
       0.1710354 , 0.06686856, 0.94715934, 0.01800794, 0.00390574,
       0.94377959, 0.14805809, 0.09467707, 0.09368938, 0.92383192,
       1.        , 0.06686921, 0.12482552, 0.15434163, 0.00896503,
       0.15745215, 0.09319132, 0.0171476 , 0.11426281, 0.17170

In [16]:
Output = list(map(lambda n: "%.1f" %(n*100), target_predicted))
Output

['14.1',
 '7.0',
 '11.2',
 '11.4',
 '9.8',
 '13.6',
 '12.9',
 '3.3',
 '1.8',
 '8.5',
 '91.7',
 '91.6',
 '85.7',
 '14.8',
 '14.5',
 '13.3',
 '9.8',
 '9.5',
 '14.5',
 '7.0',
 '13.3',
 '92.4',
 '6.7',
 '13.9',
 '16.5',
 '7.8',
 '11.4',
 '14.8',
 '15.3',
 '10.9',
 '7.0',
 '16.7',
 '8.8',
 '9.4',
 '12.4',
 '4.5',
 '85.8',
 '12.3',
 '8.1',
 '12.5',
 '11.1',
 '11.2',
 '9.5',
 '8.7',
 '0.0',
 '15.6',
 '93.1',
 '9.8',
 '6.8',
 '9.0',
 '11.2',
 '16.6',
 '13.3',
 '1.7',
 '90.1',
 '17.1',
 '6.7',
 '94.7',
 '1.8',
 '0.4',
 '94.4',
 '14.8',
 '9.5',
 '9.4',
 '92.4',
 '100.0',
 '6.7',
 '12.5',
 '15.4',
 '0.9',
 '15.7',
 '9.3',
 '1.7',
 '11.4',
 '17.2',
 '8.7',
 '14.1',
 '8.5',
 '89.4',
 '7.9',
 '12.5',
 '0.4',
 '11.3',
 '94.3',
 '92.5',
 '13.6',
 '15.3',
 '14.5',
 '14.7',
 '14.5',
 '12.3',
 '8.7',
 '8.7',
 '2.0',
 '11.1',
 '88.4',
 '8.1',
 '17.2',
 '7.0',
 '93.4',
 '2.0',
 '9.3',
 '88.4',
 '6.7',
 '1.0',
 '11.1',
 '8.1',
 '11.3',
 '6.8',
 '8.8',
 '2.0',
 '95.0',
 '9.8',
 '1.8',
 '11.1',
 '11.2',
 '14.

In [17]:
mean_absolute_error_models_dict['svr'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['svr'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['svr'] = r2_score(target_test, target_predicted)

In [18]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['svr']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['svr']))
print("R-Square Error: {}".format(r2_score_models_dict['svr']))

Mean Absolute Error: 0.1004032936138721
Mean Squared Error: 0.012068107717516763
R-Square Error: 0.9072949907154394


### Linear Regression

In [19]:
LinearRegression_model = LinearRegression()
LinearRegression_model.fit(features_train, target_train)

LinearRegression()

In [20]:
target_predicted = LinearRegression_model.predict(features_test) 
target_predicted

array([ 2.79370540e-16,  3.67047083e-16,  5.80409689e-16, -5.80931417e-16,
        1.96053178e-16, -1.15718595e-15,  1.43670176e-15, -5.86289405e-16,
       -1.10573239e-16, -6.30130747e-17,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  5.80938961e-16,  9.38769152e-17,  1.86508458e-16,
       -1.05171679e-15,  1.06686790e-15, -2.57609294e-16, -2.39160028e-15,
       -1.83310220e-17,  1.00000000e+00,  1.94568942e-15, -1.18414696e-15,
        3.20690982e-15, -1.12592362e-15, -3.76091937e-16, -9.32879208e-16,
        5.63143757e-16, -5.36238547e-16,  2.69924443e-15,  1.33858399e-15,
       -1.40986731e-15,  1.11710634e-15, -2.37267756e-15, -1.61933306e-15,
        1.00000000e+00,  6.49850454e-16, -3.17885632e-16,  8.81897412e-16,
       -1.53160579e-15, -1.34308744e-15, -3.05091018e-16,  2.11777683e-15,
       -7.47511223e-16, -2.63290792e-16,  1.00000000e+00, -2.77037444e-15,
        6.97756433e-16, -6.93256559e-16,  3.75570209e-16,  1.52232688e-15,
        1.90516611e-15,  

In [21]:
sklearn.preprocessing.minmax_scale(target_predicted, feature_range=(0, 1), axis=0, copy=False)

array([3.40789550e-15, 3.49557205e-15, 3.70893465e-15, 2.54759355e-15,
       3.32457814e-15, 1.97133902e-15, 4.56522672e-15, 2.54223556e-15,
       3.01795172e-15, 3.06551189e-15, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 3.70946392e-15, 3.22240188e-15, 3.31503342e-15,
       2.07680818e-15, 4.19539287e-15, 2.87091567e-15, 7.36924688e-16,
       3.11019394e-15, 1.00000000e+00, 5.07421438e-15, 1.94437800e-15,
       6.33543479e-15, 2.00260134e-15, 2.75243303e-15, 2.19564576e-15,
       3.69166872e-15, 2.59228642e-15, 5.82776940e-15, 4.46710895e-15,
       1.71865765e-15, 4.24563131e-15, 7.55847404e-16, 1.50919190e-15,
       1.00000000e+00, 3.77837542e-15, 2.81063933e-15, 4.01042238e-15,
       1.59691917e-15, 1.78543752e-15, 2.82343395e-15, 5.24630179e-15,
       2.38101374e-15, 2.86523417e-15, 1.00000000e+00, 3.58150528e-16,
       3.82628140e-15, 2.43526840e-15, 3.50409517e-15, 4.65085185e-15,
       5.03369107e-15, 3.91823394e-15, 1.00000000e+00, 3.08777790e-15,
      

In [22]:
Output = list(map(lambda n: "%.1f" %(n*100), target_predicted))
Output

['0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '100.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '100.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '10

In [23]:
mean_absolute_error_models_dict['lr'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['lr'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['lr'] = r2_score(target_test, target_predicted)

In [24]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['lr']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['lr']))
print("R-Square Error: {}".format(r2_score_models_dict['lr']))

Mean Absolute Error: 3.3562207570253686e-15
Mean Squared Error: 1.3435797773045497e-29
R-Square Error: 1.0


### Decision Tree Regression

In [25]:
DecisionTreeRegressor_model = DecisionTreeRegressor()
DecisionTreeRegressor_model.fit(features_train, target_train)

DecisionTreeRegressor()

In [26]:
target_predicted = DecisionTreeRegressor_model.predict(features_test)
target_predicted

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0.])

In [27]:
mean_absolute_error_models_dict['dtr'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['dtr'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['dtr'] = r2_score(target_test, target_predicted)

In [28]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['lr']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['lr']))
print("R-Square Error: {}".format(r2_score_models_dict['lr']))

Mean Absolute Error: 3.3562207570253686e-15
Mean Squared Error: 1.3435797773045497e-29
R-Square Error: 1.0


### Voting Regressor (GradientBoostRegressor, RandomForestRegressor, LinearRegression)¶ 

In [29]:
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(random_state=1)
reg3 = LinearRegression()
VotingRegressor_model = VotingRegressor(estimators=[('gbr', reg1), ('rfr', reg2), ('lr', reg3)])
VotingRegressor_model.fit(features_train, target_train)

VotingRegressor(estimators=[('gbr', GradientBoostingRegressor(random_state=1)),
                            ('rfr', RandomForestRegressor(random_state=1)),
                            ('lr', LinearRegression())])

In [30]:
target_predicted = VotingRegressor_model.predict(features_test)
target_predicted

array([1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 9.99993058e-01, 9.99993058e-01,
       9.99993058e-01, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 9.99993058e-01, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       9.99993058e-01, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 9.99993058e-01, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 1.91208818e-06, 1.91208818e-06,
       1.91208818e-06, 1.91208818e-06, 9.99993058e-01, 1.00019121e-02,
      

In [31]:
mean_absolute_error_models_dict['vreg'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['vreg'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['vreg'] = r2_score(target_test, target_predicted)

In [32]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['vreg']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['vreg']))
print("R-Square Error: {}".format(r2_score_models_dict['vreg']))

Mean Absolute Error: 0.00014254601623739274
Mean Squared Error: 1.3991467554896725e-06
R-Square Error: 0.9999892520090147


### Voting Regressor (GradientBoostRegressor, RandomForestRegressor, LinearRegression, SVR)

In [33]:
reg1 = GradientBoostingRegressor(random_state = 1)
reg2 = RandomForestRegressor(random_state = 1)
reg3 = LinearRegression()
reg4 = SVR(kernel = 'linear')
ereg = VotingRegressor(estimators=[('gbr', reg1), ('rfr', reg2), ('lr', reg3), ('svr', reg4)])
ereg.fit(features_train, target_train)

VotingRegressor(estimators=[('gbr', GradientBoostingRegressor(random_state=1)),
                            ('rfr', RandomForestRegressor(random_state=1)),
                            ('lr', LinearRegression()),
                            ('svr', SVR(kernel='linear'))])

In [34]:
target_predicted = ereg.predict(features_test)
target_predicted

array([ 1.61423259e-02, -4.62722261e-03,  7.79644414e-03,  8.31198145e-03,
        3.68058516e-03,  1.45494902e-02,  1.26129421e-02, -1.54603483e-02,
       -1.97515571e-02, -2.83163408e-04,  9.92335511e-01,  9.92028923e-01,
        9.74965611e-01,  1.81654667e-02,  1.73976016e-02,  1.38170238e-02,
        3.48509527e-03,  2.56149188e-03,  1.71387333e-02, -4.63280425e-03,
        1.38175310e-02,  9.94342205e-01, -5.46659805e-03,  1.55793740e-02,
        2.30773950e-02, -2.34112229e-03,  8.31147420e-03,  1.81647706e-02,
        1.94773166e-02,  6.82447416e-03, -4.66760404e-03,  2.38312205e-02,
        7.26683803e-04,  2.31381423e-03,  1.12205139e-02, -1.19133643e-02,
        9.75106718e-01,  1.08957829e-02, -1.48989439e-03,  1.13918180e-02,
        7.29269851e-03,  7.79676247e-03,  2.69853755e-03,  3.01910082e-04,
       -2.50026194e-02,  2.03747672e-02,  9.96312117e-01,  3.48490635e-03,
       -5.14296102e-03,  1.27109853e-03,  7.79695139e-03,  2.33445551e-02,
        1.38172127e-02, -

In [35]:
print("Mean Absolute Error: {}".format(mean_absolute_error(target_test, target_predicted)))
print("Mean Squared Error: {}".format(mean_squared_error(target_test, target_predicted)))
print("R-Square Error: {}".format(r2_score(target_test, target_predicted)))

Mean Absolute Error: 0.01172080861723924
Mean Squared Error: 0.00020475315137957485
R-Square Error: 0.9984271235189478


In [36]:
sklearn.preprocessing.minmax_scale(target_predicted, feature_range=(0, 1), axis=0, copy=False)

array([0.03950351, 0.01956254, 0.03149057, 0.03198555, 0.02753891,
       0.03797421, 0.03611492, 0.00916159, 0.00504158, 0.0237333 ,
       0.97675235, 0.97645799, 0.96007541, 0.04144593, 0.0407087 ,
       0.03727097, 0.02735122, 0.02646447, 0.04046016, 0.01955718,
       0.03727146, 0.97867899, 0.01875665, 0.03896301, 0.04616191,
       0.02175744, 0.03198506, 0.04144527, 0.04270545, 0.03055738,
       0.01952377, 0.04688566, 0.02470286, 0.02622667, 0.03477804,
       0.01256707, 0.96021088, 0.03446627, 0.02257471, 0.03494251,
       0.03100693, 0.03149088, 0.02659604, 0.02429503, 0.        ,
       0.0435671 , 0.98057031, 0.02735104, 0.01906738, 0.02522555,
       0.03149106, 0.04641841, 0.03727115, 0.00480074, 0.97217755,
       0.05507821, 0.01871831, 0.98520847, 0.00504091, 0.00109332,
       0.98426239, 0.04144545, 0.02650266, 0.02622618, 0.9786785 ,
       1.        , 0.01871849, 0.03494203, 0.04320438, 0.00250955,
       0.0440751 , 0.02608676, 0.00480008, 0.03198524, 0.04806

In [37]:
mean_absolute_error_models_dict['ereg'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['ereg'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['ereg'] = r2_score(target_test, target_predicted)

In [38]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['ereg']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['ereg']))
print("R-Square Error: {}".format(r2_score_models_dict['ereg']))

Mean Absolute Error: 0.028206297588261525
Mean Squared Error: 0.0009560147107792575
R-Square Error: 0.9926560688126502


### Gradient Boosting Regressor

In [39]:
GradientBoostingRegressor_model = GradientBoostingRegressor(random_state = 1)
GradientBoostingRegressor_model.fit(features_train, target_train)

GradientBoostingRegressor(random_state=1)

In [40]:
target_predicted = GradientBoostingRegressor_model.predict(features_test)
target_predicted

array([5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 9.99979175e-01, 9.99979175e-01,
       9.99979175e-01, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 9.99979175e-01, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       9.99979175e-01, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 9.99979175e-01, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 5.73626455e-06, 5.73626455e-06,
       5.73626455e-06, 5.73626455e-06, 9.99979175e-01, 5.73626455e-06,
      

In [41]:
mean_absolute_error_models_dict['gbr'] = mean_absolute_error(target_test, target_predicted)
mean_squared_error_models_dict['gbr'] = mean_squared_error(target_test, target_predicted)
r2_score_models_dict['gbr'] = r2_score(target_test, target_predicted)

In [42]:
print("Mean Absolute Error: {}".format(mean_absolute_error_models_dict['gbr']))
print("Mean Squared Error: {}".format(mean_squared_error_models_dict['gbr']))
print("R-Square Error: {}".format(r2_score_models_dict['gbr']))

Mean Absolute Error: 8.057629131651373e-06
Mean Squared Error: 9.456342162726664e-11
R-Square Error: 0.999999999273581


In [43]:
Output = list(map(lambda n: "%.1f" %n, target_predicted))
Output

['0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',


## Comparison

In [44]:
mean_absolute_error_models_dict

{'svr': 0.1004032936138721,
 'lr': 3.3562207570253686e-15,
 'dtr': 0.0,
 'vreg': 0.00014254601623739274,
 'ereg': 0.028206297588261525,
 'gbr': 8.057629131651373e-06}

In [45]:
mean_squared_error_models_dict

{'svr': 0.012068107717516763,
 'lr': 1.3435797773045497e-29,
 'dtr': 0.0,
 'vreg': 1.3991467554896725e-06,
 'ereg': 0.0009560147107792575,
 'gbr': 9.456342162726664e-11}

In [46]:
r2_score_models_dict

{'svr': 0.9072949907154394,
 'lr': 1.0,
 'dtr': 1.0,
 'vreg': 0.9999892520090147,
 'ereg': 0.9926560688126502,
 'gbr': 0.999999999273581}